# Факторизация матриц
## Содержание
1. [$\mathbf{LU}$ - разложение](#lu_decomposition)
2. [Разложение Холецкого ($\mathbf{LL^T}$ - разложение)](matrix_decomposition_cholesk.ipynb)
3. $\mathbf{QR}$ - разложение
4. Спектральное разложение
5. Сингулярное разложение

<a id='lu_decomposition'></a>
## $\mathbf{LU}$ - разложение
$\mathbf{LU}$ - разложением называется разложение **квадратной** матрицы на произведение нижнетреугольной и верхнетреугольной матриц
$$
\mathbf{A = LU}\\
\begin{bmatrix}
        a_{11} & a_{12} & a_{13} & \ldots & a_{1n} \\
        a_{21} & a_{22} & a_{23} & \ldots & a_{2n} \\
        a_{31} & a_{32} & a_{33} & \ldots & a_{3n} \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        a_{n1} & a_{n2} & a_{n3} & \ldots & a_{nn} \\ 
\end{bmatrix}
 = 
\begin{bmatrix}
        1 & 0 & 0 & \ldots & 0 \\
        l_{21} & 1 & 0 & \ldots & 0 \\
        l_{31} & l_{32} & 1 & \ldots & 0 \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        l_{n1} & l_{n2} & l_{n3} & \ldots & 1 \\ 
\end{bmatrix}
\begin{bmatrix}
        u_{11}  & u_{12}        & u_{13}        & \ldots        & u_{1n} \\
        0       & u_{22}        & u_{23}        & \ldots        & u_{2n} \\
        0       & 0             & u_{33}        & \ldots        & u_{3n} \\
        \vdots  & \vdots        & \vdots        & \ddots        & \vdots \\
        0       & 0             & 0             & \ldots        & u_{nn} \\ 
\end{bmatrix}
$$


Если мы расспишем произведение $\mathbf{LU}$, то получится что матрица $\mathbf{A}$ равна:
$$
\begin{bmatrix}
        a_{11} & a_{12} & a_{13} & \ldots & a_{1n} \\
        a_{21} & a_{22} & a_{23} & \ldots & a_{2n} \\
        a_{31} & a_{32} & a_{33} & \ldots & a_{3n} \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        a_{n1} & a_{n2} & a_{n3} & \ldots & a_{nn} \\ 
\end{bmatrix}
 = 
 \begin{bmatrix}
        u_{11}          & u_{22}                        & u_{23}                                        & \ldots & u_{1n} \\
        l_{21}u_{11}    & l_{21}u_{12} + u_{22}         & l_{21}u_{13} + u_{23}                         & \ldots & l_{21}u_{1n} + u_{2n} \\
        l_{31}u_{11}    & l_{31}u_{12} + l_{32}u_{22}   & l_{31}u_{13} + l_{32}u_{23} + u_{33}          & \ldots & l_{31}u_{1n} + l_{32}u_{2n} + u_{3n} \\
        \vdots          & \vdots                        & \vdots                                        & \ddots & \vdots \\
        l_{n1}u_{11}    & l_{n1}u_{12} + l_{n2}u_{22}   & l_{n1}u_{13} + l_{n2}u_{23} + l_{n3}u_{33}    & \ldots & \left( \sum_{i=1}^{n-1}{l_{ni}u_{in}} \right) + u_{nn} \\ 

 \end{bmatrix}

 $$

Из полученного выражения путем индукции можно записать выражения для элементов матриц $\mathbf{L}$ и $\mathbf{U}$
$$
    i = 1..n; \ j = i..n\\
    u_{ij} = a_{ij} - \sum_{k=1}^{i-1}{l_{ik}u_{kj}}\\
    l_{ii} = 1; \quad
    l_{ji} = \left( u_{jj} \right)^{-1} \left( a_{ji} - \sum_{k = 1}^{i-1}{l_{jk}u_{ki}} \right)
$$

Используя приведенные формулы напишите функцию для расчета $\mathbf{LU}$ - разложения

In [2]:
function myLU(A::AbstractMatrix{<:Number})::Tuple{Matrix{Float64},Matrix{Float64}}
    
    N = size(A,1);
    L = zeros(Float64, N, N); U = zeros(Float64, N, N);
    for i in 1:N
        L[i,i] = 1; 
        for j in i:N
            U[i,j] = A[i,j] - L[i, 1:i-1]' * U[ 1:i-1, j];
            L[j,i] = (A[j,i] - L[j, 1:i-1]' * U[1:i-1, i]) / U[i,i]
        end
    end
    return (L, U)
end

myLU (generic function with 1 method)

In [51]:
A = rand(10,10)
L, U = myLU(A);
@assert A ≈ L*U

$\mathbf{LU}$ - разложение полезно для следующих задач:
1. Несложно заметить, что посколько матрицы треугольные, то их определитель будет равен произведению диагональных элементов, а учитывая что $l_{ii}=1$, то $\det(\mathbf{L})=1$, следовательно
$$
\det(\mathbf{A}) = \det(\mathbf{U}) = \prod_{i=1}^{n}{u_{ii}}
$$
2. Имея $\mathbf{LU}$ факторизацию матрицы довольно легко получить решение уравнения вида
$$
\mathbf{Ax}=\mathbf{b}
$$
используя метод решения уравнений с треугольной матрицей (обратный проход метода Гаусса):
$$
1.\ \mathbf{Ly}=\mathbf{b}\\
\\
2.\ \mathbf{Ux}=\mathbf{y}
$$

Реализуйте функцию решения системы с тругольной матрицей

In [5]:
function solvetriang(T::AbstractMatrix{<:Number}, b::AbstractVector{<:Number}, type::Symbol)::Vector{Float64}
    N = size(b,1);
    x = Vector{Float64}(undef, N);
    if type == :Upper
        for i in N:-1:1
            x[i] = (b[i] - x[i+1:end]' * T[i, i+1:end])/T[i,i]
        end
    elseif type == :Lower
        for i in 1:N
            x[i] = (b[i] - x[1:i-1]' * T[i, 1:i-1])/T[i,i];
        end
    else
        throw(Error("Unknown type of matrix"))
    end
    return x;
end

solvetriang (generic function with 1 method)

Примените реализованные функции для решения системы со случайными матрицами

In [10]:
A = rand(10,10); b = rand(10);
L, U = myLU(A);
y = solvetriang(L,b,:Lower);
x = solvetriang(U,y,:Upper);
@assert x ≈ A\b